<a href="https://colab.research.google.com/github/jasonericr/dissertation/blob/main/dissertation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Research Steps**
1. Load Data
2. Concantenate Financial Statement Data into 1 dateframe
3. Clean Data
<br/>
    a. Check/impute for missing values
    b.

In [ ]:
# for feature selection using reinforcement learning
!pip install FSRLearning

In [ ]:
!pip install openpyxl

In [ ]:
# import needed libraries
from google.colab import files
import io
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
from openpyxl import load_workbook
import FSRLearning
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from statsmodels.tsa.seasonal import STL

import torch.nn as torch

#Update pandas display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 500)

In [ ]:
# financial statement data files to upload
uploaded = files.upload()

Saving aapl-financials.xlsx to aapl-financials.xlsx
Saving brk.a-financials.xlsx to brk.a-financials.xlsx
Saving cvs-financials.xlsx to cvs-financials.xlsx
Saving f-financials.xlsx to f-financials.xlsx
Saving gm-financials.xlsx to gm-financials.xlsx
Saving hd-financials.xlsx to hd-financials.xlsx
Saving jnj-financials.xlsx to jnj-financials.xlsx
Saving jpm-financials.xlsx to jpm-financials.xlsx
Saving ko-financials.xlsx to ko-financials.xlsx
Saving kr-financials.xlsx to kr-financials.xlsx
Saving mck-financials.xlsx to mck-financials.xlsx
Saving msft-financials.xlsx to msft-financials.xlsx
Saving pg-financials.xlsx to pg-financials.xlsx
Saving vz-financials.xlsx to vz-financials.xlsx
Saving wmt-financials.xlsx to wmt-financials.xlsx
Saving xom-financials.xlsx to xom-financials.xlsx


In [ ]:
# create an empty dataframe
df = pd.DataFrame()

# loop through all fin statement excel files and load to df
for file in uploaded.keys():
  # create temp df to load each individual stock
  temp_df = pd.DataFrame()

  # upload file from google docs and transpose; upload by each FS type
  xlsx_file = io.BytesIO(uploaded.get(file))
  income_stmt = pd.read_excel(xlsx_file, 'Income-Quarterly', header=None)
  income_stmt_t = income_stmt.set_index([0]).T.reset_index()
  income_stmt_t = income_stmt_t.drop(columns=['EPS (Diluted)'])

  income_stmt_ttm = pd.read_excel(xlsx_file, 'Income-TTM',  header=None)
  income_stmt_ttm_t = income_stmt_ttm.set_index([0]).T.reset_index()
  income_stmt_ttm_t = income_stmt_ttm_t[['index', 'EPS (Diluted)']]
  income_stmt_ttm_t.rename(columns={'index':'index_v'}, inplace=True)

  bs_stmt = pd.read_excel(xlsx_file, 'Balance-Sheet-Quarterly', header=None)
  bs_stmt_t = bs_stmt.set_index([0]).T.reset_index()
  bs_stmt_t.rename(columns={'index':'index_x', 'Date':'date_x'}, inplace=True)

  cf_stmt = pd.read_excel(xlsx_file, 'Cash-Flow-Quarterly', header=None)
  cf_stmt_t = cf_stmt.set_index([0]).T.reset_index()
  cf_stmt_t.rename(columns={'index':'index_y', 'Date':'date_y'}, inplace=True)

  ratios_stmt = pd.read_excel(xlsx_file, 'Ratios-Quarterly', header=None)
  ratios_stmt_t = ratios_stmt.set_index([0]).T.reset_index()
  ratios_stmt_t.rename(columns={'index':'index_z', 'Date':'date_z'}, inplace=True)

  # concatenate all 4 different fin statements into temp df and add field for ticker symbol
  temp_df = pd.concat([income_stmt_t, income_stmt_ttm_t, bs_stmt_t, cf_stmt_t, ratios_stmt_t], axis=1)
  temp_df = temp_df.drop(columns=['index_v', 'index_x', 'index_y', 'index_z', 'date_x', 'date_y', 'date_z'])
  temp_df['ticker_symbol'] = file.replace('-financials.xlsx', '')

  if df.empty:
    df = temp_df
  else:
    df = pd.concat([df, temp_df], axis=0)



In [ ]:
# calculate stock price at each time interval from EPS and PE Ratio
df['share_price'] = df['EPS (Diluted)'] * df['PE Ratio']
df['share_price'] = df['share_price'].apply(lambda x: np.round(x, 2))

In [ ]:
# check all ticker symbols in data
df.ticker_symbol.unique()

array(['aapl', 'brk.a', 'cvs', 'f', 'gm', 'hd', 'jnj', 'jpm', 'ko', 'kr',
       'mck', 'msft', 'pg', 'vz', 'wmt', 'xom'], dtype=object)

In [ ]:
## Data Cleaning

In [ ]:
# Ticker symbols and their amount of missing share prices
df.groupby(['ticker_symbol']).agg(null_vals=('share_price', lambda x:x.isnull().sum()))

,null_vals
ticker_symbol,
aapl,5
brk.a,87
cvs,8
f,5
gm,5
hd,5
jnj,5
jpm,5
ko,19


In [ ]:
# Determine which indices have most missing values
df[df['share_price'].isnull()][['index']].value_counts()

,count
index,
114.0,8
113.0,8
116.0,8
115.0,8
109.0,5
111.0,5
110.0,4
122.0,4
124.0,4


In [ ]:
# Replace missing share prices with feed-forward price (price from previous period reported)
df_sorted = df.sort_values(by=['ticker_symbol', 'index'], ascending=[True, True])

# For missing share prices, fill using previous period share price
df_sorted['share_price'] = df_sorted['share_price'].ffill()

In [ ]:
# Verify that the share price field has no NULL values
assert df_sorted[df_sorted['share_price'].isnull()].shape[0] == 0, 'Not all share prices values are zero. Check the forward-fill.'

In [ ]:
# remove Date field as no longer needed given Index provides chronological variable
df_no_date = df_sorted.drop(columns=['Date'])

# Drop columns missing more than 20% of data
no_missing_data_mask = ((df_no_date.isnull().sum()/df_no_date.shape[0]) < 0.15)

# Keep only columns with 80% or more of data
df_filtered = df_no_date[list(df_no_date.columns[no_missing_data_mask])]
df_filtered.head()

,index,Revenue,Revenue Growth,Cost of Revenue,Gross Profit,"Selling, General & Admin",Research & Development,Operating Expenses,Other Operating Expenses,Operating Income,Interest Expense / Income,Other Expense / Income,Pretax Income,Income Tax,Net Income,Net Income,Preferred Dividends,Net Income Common,Shares Outstanding (Basic),Shares Change,EPS (Basic),Free Cash Flow Per Share,Free Cash Flow Per Share,Dividend Per Share,Gross Margin,Operating Margin,Profit Margin,Free Cash Flow Margin,Free Cash Flow Margin,Effective Tax Rate,EBITDA,EBITDA Margin,Depreciation & Amortization,Depreciation & Amortization,EBIT,EBIT Margin,EPS (Diluted),Cash & Equivalents,Cash & Cash Equivalents,Cash Growth,Receivables,Inventory,"Property, Plant & Equipment",Goodwill and Intangibles,Total Assets,Accounts Payable,Deferred Revenue,Total Liabilities,Total Debt,Debt Growth,Retained Earnings,Comprehensive Income,Shareholders Equity,Net Cash / Debt,Book Value Per Share,Net Income,Net Income,Depreciation & Amortization,Depreciation & Amortization,Share-Based Compensation,Other Operating Activities,Operating Cash Flow,Capital Expenditures,Acquisitions,Change in Investments,Other Investing Activities,Investing Cash Flow,Dividends Paid,Share Issuance / Repurchase,Debt Issued / Paid,Financing Cash Flow,Net Cash Flow,Free Cash Flow,Free Cash Flow Margin,Free Cash Flow Margin,Free Cash Flow Per Share,Free Cash Flow Per Share,Market Capitalization,Market Cap Growth,Enterprise Value,PE Ratio,PS Ratio,PB Ratio,P/FCF Ratio,P/OCF Ratio,Debt/Equity,Return on Invested Capital (ROIC),Dividend Yield,Payout Ratio,Buyback Yield,Total Return,ticker_symbol,share_price
0,1.0,94930,0.060694,51051,43879,6523,7765,14288,0,29591,0,-19,29610,14874,14736,14736,0,14736,15171.991,-0.027408,0.98,1.575,1.575,0.25,0.462225,0.311714,0.15523,0.251796,0.251796,0.50233,32521,0.342579,2911,2911,29610,0.311914,6.08,29943,156650,-0.033615,66243,7286,45680,0,364980,68960,8249,308030,106629,-0.040139,-19154,-7172,56950,50021,3.754,14736,14736,2911,2911,2858,6306,26811,-2908,0,4544,-191,1445,-3804,-25083,4387,-24948,3308,23903,0.251796,0.251796,1.575,1.575,3463350.36723,0.29387,3413329.36723,36.948,8.857,60.814,31.830217,29.287385,1.872327,0.571687,0.004,0.255,0.027408,0.031408,aapl,224.64
1,2.0,85777,0.048657,46099,39678,6320,8006,14326,0,25352,0,-142,25494,4046,21448,21448,0,21448,15287.521,-0.027058,1.4,1.747,1.747,0.25,0.462572,0.295557,0.250044,0.311354,0.311354,0.158704,28344,0.330438,2850,2850,25494,0.297213,6.57,25565,153041,-0.081072,43172,6165,44502,0,331612,47574,8053,264904,101304,-0.072987,-4726,-8416,66708,51737,4.364,21448,21448,2850,2850,2869,1691,28858,-2151,0,2412,-388,-127,-3895,-26522,-3253,-36017,-7286,26707,0.311354,0.311354,1.747,1.747,3229664.35084,0.058595,3177927.35084,31.677,8.376,48.415,30.953568,28.570734,1.518618,0.605438,0.005,0.179,0.027058,0.032058,aapl,208.12
2,3.0,90753,-0.043053,48482,42271,6468,7903,14371,0,27900,0,-158,28058,4422,23636,23636,0,23636,15405.856,-0.024127,1.53,1.343,1.343,0.24,0.465781,0.307428,0.260443,0.228026,0.228026,0.157602,30894,0.340418,2836,2836,28058,0.309169,6.43,32695,162337,-0.024024,41150,6232,43546,0,337411,45753,8012,263217,104590,-0.045842,4339,-8960,74194,57747,4.816,23636,23636,2836,2836,2964,-6746,22690,-1996,0,2131,-445,-310,-3710,-23205,-3148,-30433,-8053,20694,0.228026,0.228026,1.343,1.343,2647973.75388,0.014923,2590226.75388,26.377,6.939,35.69,25.981159,23.949909,1.409683,0.562159,0.006,0.157,0.024127,0.030127,aapl,169.60
3,4.0,119575,0.020665,64720,54855,6786,7696,14482,0,40373,0,50,40323,6407,33916,33916,0,33916,15509.763,-0.023758,2.19,2.418,2.418,0.24,0.45875,0.337637,0.283638,0.313636,0.313636,0.158892,43171,0.361037,2848,2848,40323,0.337219,6.42,40760,172575,0.043064,50102,6511,43666,0,353514,58146,8264,279414,108040,-0.02763,8242,-9378,74100,64535,4.778,33916,33916,2848,2848,2997,134,39895,-2392,0,4603,-284,1927,-3825,-20139,-3984,-30585,11237,37503,0.313636,0.313636,2.418,2.418,2994371.34256,0.448697,2929836.342

In [ ]:
df_filtered.isnull().sum()/df_filtered.shape[0]<=0

,0
0,
index,False
Revenue,False
Revenue Growth,False
Cost of Revenue,False
Gross Profit,False
"Selling, General & Admin",False
Research & Development,False
Operating Expenses,False
Other Operating Expenses,False


In [ ]:
from sklearn.ensemble import HistGradientBoostingRegressor

In [ ]:
# Use MICE technique to fill missing values

# First, separate ticker_symbol fields given its a text field and will not work under MICE
#df_filtered_val = df_filtered.drop(columns=['ticker_symbol', 'index'], axis=1)
#ticker_df = df_filtered[['ticker_symbol']]



In [ ]:
# clean column naming
df_filtered.columns = [col.lower() for col in df_filtered.columns]
df_filtered.columns = [col.replace(" ", "_") for col in df_filtered.columns]
df_filtered.columns = [col.replace(",", "") for col in df_filtered.columns]
df_filtered.columns = [col.replace("&", "") for col in df_filtered.columns]
df_filtered.columns = [col.replace("(", "") for col in df_filtered.columns]
df_filtered.columns = [col.replace(")", "") for col in df_filtered.columns]
df_filtered.columns = [col.replace("/", "") for col in df_filtered.columns]

df_copy = df_filtered.loc[:,~df_filtered.columns.duplicated()].copy()
df_copy.head()

,index,revenue,revenue_growth,cost_of_revenue,gross_profit,selling_general__admin,research__development,operating_expenses,other_operating_expenses,operating_income,interest_expense__income,other_expense__income,pretax_income,income_tax,net_income,preferred_dividends,net_income_common,shares_outstanding_basic,shares_change,eps_basic,free_cash_flow_per_share,dividend_per_share,gross_margin,operating_margin,profit_margin,free_cash_flow_margin,effective_tax_rate,ebitda,ebitda_margin,depreciation__amortization,ebit,ebit_margin,eps_diluted,cash__equivalents,cash__cash_equivalents,cash_growth,receivables,inventory,property_plant__equipment,goodwill_and_intangibles,total_assets,accounts_payable,deferred_revenue,total_liabilities,total_debt,debt_growth,retained_earnings,comprehensive_income,shareholders_equity,net_cash__debt,book_value_per_share,share-based_compensation,other_operating_activities,operating_cash_flow,capital_expenditures,acquisitions,change_in_investments,other_investing_activities,investing_cash_flow,dividends_paid,share_issuance__repurchase,debt_issued__paid,financing_cash_flow,net_cash_flow,free_cash_flow,market_capitalization,market_cap_growth,enterprise_value,pe_ratio,ps_ratio,pb_ratio,pfcf_ratio,pocf_ratio,debtequity,return_on_invested_capital_roic,dividend_yield,payout_ratio,buyback_yield,total_return,ticker_symbol,share_price
0,1.0,94930,0.060694,51051,43879,6523,7765,14288,0,29591,0,-19,29610,14874,14736,0,14736,15171.991,-0.027408,0.98,1.575,0.25,0.462225,0.311714,0.15523,0.251796,0.50233,32521,0.342579,2911,29610,0.311914,6.08,29943,156650,-0.033615,66243,7286,45680,0,364980,68960,8249,308030,106629,-0.040139,-19154,-7172,56950,50021,3.754,2858,6306,26811,-2908,0,4544,-191,1445,-3804,-25083,4387,-24948,3308,23903,3463350.36723,0.29387,3413329.36723,36.948,8.857,60.814,31.830217,29.287385,1.872327,0.571687,0.004,0.255,0.027408,0.031408,aapl,224.64
1,2.0,85777,0.048657,46099,39678,6320,8006,14326,0,25352,0,-142,25494,4046,21448,0,21448,15287.521,-0.027058,1.4,1.747,0.25,0.462572,0.295557,0.250044,0.311354,0.158704,28344,0.330438,2850,25494,0.297213,6.57,25565,153041,-0.081072,43172,6165,44502,0,331612,47574,8053,264904,101304,-0.072987,-4726,-8416,66708,51737,4.364,2869,1691,28858,-2151,0,2412,-388,-127,-3895,-26522,-3253,-36017,-7286,26707,3229664.35084,0.058595,3177927.35084,31.677,8.376,48.415,30.953568,28.570734,1.518618,0.605438,0.005,0.179,0.027058,0.032058,aapl,208.12
2,3.0,90753,-0.043053,48482,42271,6468,7903,14371,0,27900,0,-158,28058,4422,23636,0,23636,15405.856,-0.024127,1.53,1.343,0.24,0.465781,0.307428,0.260443,0.228026,0.157602,30894,0.340418,2836,28058,0.309169,6.43,32695,162337,-0.024024,41150,6232,43546,0,337411,45753,8012,263217,104590,-0.045842,4339,-8960,74194,57747,4.816,2964,-6746,22690,-1996,0,2131,-445,-310,-3710,-23205,-3148,-30433,-8053,20694,2647973.75388,0.014923,2590226.75388,26.377,6.939,35.69,25.981159,23.949909,1.409683,0.562159,0.006,0.157,0.024127,0.030127,aapl,169.60
3,4.0,119575,0.020665,64720,54855,6786,7696,14482,0,40373,0,50,40323,6407,33916,0,33916,15509.763,-0.023758,2.19,2.418,0.24,0.45875,0.337637,0.283638,0.313636,0.158892,43171,0.361037,2848,40323,0.337219,6.42,40760,172575,0.043064,50102,6511,43666,0,353514,58146,8264,279414,108040,-0.02763,8242,-9378,74100,64535,4.778,2997,134,39895,-2392,0,4603,-284,1927,-3825,-20139,-3984,-30585,11237,37503,2994371.34256,0.448697,2929836.34256,29.673,7.763,40.41,28.019083,25.717549,1.45803,0.55511,0.005,0.11,0.023758,0.028758,aapl,190.50
4,5.0,89498,-0.007188,49071,40427,6151,7307,13458,0,26969,0,-29,26998,4042,22956,0,22956,15599.433,-0.027674,1.47,1.246,0.24,0.451708,0.301336,0.256497,0.217156,0.149715,29651,0.331303,2653,26998,0.30166,6.13,29965,162099,-0.041453,60985,6331,43715,0,352583,62611,8061,290437,111088,-0.074799,-214,-11452,62146,51011,3.984,2625,-6636,21598,-2163,0,5141,-584,2394,-3758,-21003,1993,-23153,839,19435,2676736.86072,0.107223,2625725.86072,27.597,6.984,43.072,26.879186,24.21444,1.787533,0.562699,0.005,0

In [ ]:

# create empty dataframe for holding imputed data
final_imputed_data_df = pd.DataFrame()

# Separate variables to interpolate from those not to interpolate [index, ticker symbol, share price]
vars_to_interpolate = df_copy.drop(columns=['index', 'ticker_symbol', 'share_price']).columns.to_list()
company_list = list(df_copy['ticker_symbol'].unique())

# Loop through all companies and all variables performing time-series imputation
for company in company_list:
  # Filter on each individual company
  df_imputed_comp = df_copy[df_copy['ticker_symbol']==company]
  df_imputed_comp_copy = df_imputed_comp.copy()
  for var in vars_to_interpolate:

    # Locate indices with missing values
    null_indices = df_imputed_comp_copy[df_imputed_comp_copy[var].isnull()].index

    # Apply Seasonl Trend Decomposition using Loess (STL)
    stl = STL(df_imputed_comp[var].interpolate(), period=df_imputed_comp_copy.shape[0])
    stl_fit = stl.fit()

    # Pull seasonal trend component
    season_comp = stl_fit.seasonal

    # Remove seasonal component from
    df_unseasoned = df_imputed_comp_copy[var] - season_comp

    # Interpolate values using deseasoned data
    df_unseasoned_imputed = df_unseasoned.interpolate(method="linear")

    # Add back seasonal component
    df_imputed_season = df_unseasoned_imputed + season_comp

    # Update the original dataframe with the imputed values
    df_imputed_comp_copy.loc[null_indices, var] = df_imputed_season[null_indices]
    df_imputed_comp_copy = df_imputed_comp_copy.dropna(subset=[var])

  if final_imputed_data_df.empty:
    final_imputed_data_df = df_imputed_comp_copy
  else:
    final_imputed_data_df = pd.concat([final_imputed_data_df, df_imputed_comp_copy], axis=0)

<ipython-input-18-3bcfb552b761>:19: FutureWarning: Series.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  stl = STL(df_imputed_comp[var].interpolate(), period=df_imputed_comp_copy.shape[0])
<ipython-input-18-3bcfb552b761>:29: FutureWarning: Series.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  df_unseasoned_imputed = df_unseasoned.interpolate(method="linear")
<ipython-input-18-3bcfb552b761>:19: FutureWarning: Series.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  stl = STL(df_imputed_comp[var].interpolate(), period=df_imputed_comp_copy.shape[0])
<ipython-input-18-3bcfb552b761>:29: FutureWarning: Series.interpolate with object dtype is deprecated and will raise in a future version. Call obj.inf

In [ ]:
final_imputed_data_df.isnull().sum()/final_imputed_data_df.shape[0]

,0
index,0.0
revenue,0.0
revenue_growth,0.0
cost_of_revenue,0.0
gross_profit,0.0
selling_general__admin,0.0
research__development,0.0
operating_expenses,0.0
other_operating_expenses,0.0
operating_income,0.0


In [ ]:
#

In [ ]:
# PCA


In [ ]:
# Feature Selction Reinforcement Learning

In [ ]:
#